In [1]:
###params
import os
try:
    os.environ['OPT']
except:
    os.environ['OPT']='validation'
opt = os.environ['OPT']

# Description

- Process the dataset to train the model
- Create the labels

to test the notebook
- papermill notebooks/feature_selection.ipynb notebooks/output/code_exec.ipynb --log-output

# Import libraries and define functions and paths

## libraries

In [2]:
import pandas as pd
import datetime
import os
import sqlalchemy as db
from sklearn.preprocessing import LabelEncoder
import pickle

## functions

In [3]:
def get_connection():
    try:
        os.environ["DB_HOST"]
    except:
        os.environ["DB_HOST"] = "localhost"



    # specify database configurations
    config = {
        'host': os.environ['DB_HOST'],
        'port': 3306,
        'user': 'mendes',
        'password': 'test',
        'database': 'test_db'
    }
    db_user = config.get('user')
    db_pwd = config.get('password')
    db_host = config.get('host')
    db_port = config.get('port')
    db_name = config.get('database')
    # specify connection string
    connection_str = f'mysql+pymysql://{db_user}:{db_pwd}@{db_host}:{db_port}/{db_name}'
    # connect to database
    engine = db.create_engine(connection_str)
    return engine

## Paths

# Connect SQL DB and transfer data

In [4]:
engine = get_connection()

## Process dataset

In [5]:
with engine.connect() as conn:
    sql = f'SELECT * FROM {opt};'
    data = pd.read_sql(sql,conn)

In [6]:
if opt=='train' or opt=='test':
    ##separate labels
    labels = data['species']

    ## drop information about timestamp
    data = data.drop(['timestamp','species','index'],axis=1)
else:
    #TEMPORARY
    ##separate labels
    labels = data['species']

    ## drop information about timestamp
    data = data.drop(['timestamp','species','index'],axis=1)

In [7]:
if opt =='train':
    ## encode the labels to numerical values
    le = LabelEncoder()
    l = le.fit_transform(labels)
    
    with open('label_encoder.p','wb') as handle:
        pickle.dump(le,handle,protocol=pickle.HIGHEST_PROTOCOL)  
elif opt=='test':
    with open('label_encoder.p','rb') as handle:
        le = pickle.load(handle)
    l=le.transform(labels)
elif opt=='validation':
    pass

In [8]:
with engine.connect() as conn:
    if opt=='train' or opt=='test':
        df = pd.concat([data,pd.Series(l,name='label')],axis=1)
        df.to_sql(f'{opt}_preprocessed',conn,index=False,if_exists='replace')
    elif opt=='validation':
        df=data
        df.to_sql(f'{opt}_preprocessed',conn,index=False,if_exists='replace')

In [9]:
print('worked feature selection')

worked feature selection
